In [74]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

resultPath = "resultFiles/featureExtractionV3_by_Jun/"

intersected_genes = []
for x in ['CD4', 'CD8', 'CD14']:
    #df_healthy = pd.read_csv(resultPath+"HealthyPatients/"+x+".Ranksum.RFECV.gene.result", index_col=0)
    df_long = pd.read_csv(resultPath+"LongDiseaseDuration/"+x+".Ranksum.RFECV.gene.result", index_col=0)
    print(x)
    print(len(df_long))


CD4
43
CD8
149
CD14
637


In [77]:
## Utils and Library for notebook
from notebook_utils.OpenKbcMSToolkit import ExtractionToolkit as exttoolkit
import itertools
def _LoadDiseaseDuration(df, meta_data, returntype='long'):
    """
    df : Expression or activation score matrix
    meta_data : meta data which contains duration and sample ID
    output: long DD samples and short DD samples by list, or healthy samples and short DD samples by list
    """
    # Sample by disease category
    sample_list, sample_category = exttoolkit.get_sample_name_by_category(dataframe=meta_data, sampleColumn='HCVB_ID', dataColname='DiseaseCourse')
    
    # Sort by disease category and exclude uknown samples
    patient_samples = [] # patient samples
    healthy_samples = [] # healthy samples
    for samples, category in zip(sample_list, sample_category):
        if category=='Healthy':
            healthy_samples = samples
        else:
            if category!='Unknown':# Excluding unknown samples
                patient_samples.append(samples)

    patient_samples = list(itertools.chain(*patient_samples)) # flatten
    patient_samples = list(set(patient_samples).intersection(df.columns.tolist())) # intersected with act score matrix
    healthy_samples = list(set(healthy_samples).intersection(df.columns.tolist())) # intersected with act score matrix
    patient_meta = meta_data.loc[meta_data['HCVB_ID'].isin(patient_samples)] # Make patient metadata

    longDD_samples, shortDD_samples = exttoolkit.get_sample_name_by_contValues(patient_meta, 'HCVB_ID', 'DiseaseDuration', 50)
    longDD_samples = list(set(longDD_samples.values.tolist()).intersection(df.columns.tolist())) # intersected with act score matrix
    shortDD_samples = list(set(shortDD_samples.values.tolist()).intersection(df.columns.tolist())) # intersected with act score matrix

    if returntype=='long':
        return longDD_samples, shortDD_samples
    elif returntype=='healthy':
        return healthy_samples, shortDD_samples

In [84]:
#df_cd4 = pd.read_csv(resultPath+"LongDiseaseDuration/CD4.Ranksum.RFECV.act.csv", index_col=0)
df_cd4 = pd.read_csv(resultPath+"LongDiseaseDuration/CD8.Ranksum.RFECV.gene.result", index_col=0)
#df_cd4 = pd.read_csv("../data/counts_normalized/IDConvertedFiles/counts_vst_CD4.converted.csv", index_col=0)
meta_data = pd.read_csv('../data/annotation_metadata/EPIC_HCvB_metadata_baseline_updated-share.csv')
long_samples, shortDD_samples = _LoadDiseaseDuration(df_cd4, meta_data, 'long')
df_cd4 = df_cd4[long_samples+shortDD_samples]
df_cd4 = df_cd4.subtract(df_cd4.median(axis=1), axis=0)


fold_change = (df_cd4[long_samples].mean(axis=1) - df_cd4[shortDD_samples].mean(axis=1)).apply(abs)
fold_change = fold_change.sort_values(ascending=False)[:40].index.tolist()
df_cd4 = df_cd4.loc[fold_change]

# Import some data to play with
X = df_cd4.T.values
y = [0]*len(long_samples)+[1]*len(shortDD_samples) # Training y
y = np.array(y)

In [85]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics


for t in list(range(45,50)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=t)
    #X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=t)

    randomState = list(range(40,60))

    test_auc = []
    val_auc = []
    for i in randomState:
        clf = RandomForestClassifier(max_depth=10, random_state=i)
        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
        #print("test auc: ",metrics.auc(fpr, tpr))
        test_auc.append(metrics.auc(fpr, tpr))
        
        #y_val_pred = clf.predict(X_val)
        #fpr, tpr, thresholds = metrics.roc_curve(y_val, y_val_pred, pos_label=1)
        #print("val auc: ",metrics.auc(fpr, tpr))
        #val_auc.append(metrics.auc(fpr, tpr))
    #print(np.mean(test_auc), np.mean(val_auc))
    print(np.mean(test_auc))

0.6107080419580418
0.5788095238095239
0.6720274390243903
0.5882142857142857
0.6155594405594405
